<a href="https://colab.research.google.com/github/amankiitg/GenAI/blob/main/LLM_Scratch_Lecture_5_6_Byte_Pair_Encoding_from_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Why does character level tokenization fail?

In [1]:
sentence = "Today, I want to start my day with a cup of coffee"

# Split on whitespace to get individual words
words = sentence.split()

# Print the list of word tokens
print("Word tokens:", words)

# Print the total number of words
print("Number of words:", len(words))


sentence = "Today, I want to start my day with a cup of coffee"

# Convert the sentence into a list of individual characters
characters = list(sentence)

# Print the list of character tokens
print("Character tokens:", characters)

# Print the total number of characters
print("Number of characters:", len(characters))



Word tokens: ['Today,', 'I', 'want', 'to', 'start', 'my', 'day', 'with', 'a', 'cup', 'of', 'coffee']
Number of words: 12
Character tokens: ['T', 'o', 'd', 'a', 'y', ',', ' ', 'I', ' ', 'w', 'a', 'n', 't', ' ', 't', 'o', ' ', 's', 't', 'a', 'r', 't', ' ', 'm', 'y', ' ', 'd', 'a', 'y', ' ', 'w', 'i', 't', 'h', ' ', 'a', ' ', 'c', 'u', 'p', ' ', 'o', 'f', ' ', 'c', 'o', 'f', 'f', 'e', 'e']
Number of characters: 50


In [10]:
from collections import Counter
s = 'RRRRRRWs Ws Ws plRplRplWs gigWic gigWic gigWic'
Counter(s)

Counter({'R': 8,
         'W': 7,
         's': 4,
         ' ': 6,
         'p': 3,
         'l': 3,
         'g': 6,
         'i': 6,
         'c': 3})

## Implementing Byte Pair Encoding (BPE) from scratch

🦇🦇🦇

### Step 1: Take raw text and tokenize into characters

In [ ]:
# making the training text longer to have more representative token statistics
# text from https://www.reedbeta.com/blog/programmers-intro-to-unicode/
text = """The Dark Knight Rises is a superhero movie released in 2012. It is the final part of Christopher Nolan’s Dark Knight trilogy, following Batman Begins and The Dark Knight. The film stars Christian Bale as Bruce Wayne/Batman, who has been retired as Batman for eight years after the events of the previous movie.

The main villain in the movie is Bane, played by Tom Hardy. Bane is a powerful and intelligent terrorist who threatens Gotham City with destruction. He forces Bruce Wayne to come out of retirement and become Batman again. Anne Hathaway plays Selina Kyle, also known as Catwoman, a skilled thief with her own agenda.

The movie is about Bruce Wayne’s struggle to overcome his physical and emotional challenges to save Gotham. It also shows themes of hope, sacrifice, and resilience. The film has many exciting action scenes, such as a plane hijack and a massive battle in Gotham.

In the end, Batman saves the city and inspires the people of Gotham. However, he is believed to have sacrificed his life. The movie ends with a twist, suggesting that Bruce Wayne is alive and has moved on to live a quiet life.

The Dark Knight Rises was a big success and is loved by many fans for its epic story, strong characters, and thrilling action.

"""

## For non-alphabet characters and for a more general purpose code, use this:
#tokens = text.encode("utf-8") # raw bytes
#tokens = list(map(int, tokens)) # convert to a list of integers in range 0..255 for convenience

# For sake of simplicity, we are only using ASCII character encoding:
tokens = [ord(ch) for ch in text]

In [ ]:
ids = list(tokens)  # copy so we don't destroy the original list


In [ ]:
print(ids)

[84, 104, 101, 32, 68, 97, 114, 107, 32, 75, 110, 105, 103, 104, 116, 32, 82, 105, 115, 101, 115, 32, 105, 115, 32, 97, 32, 115, 117, 112, 101, 114, 104, 101, 114, 111, 32, 109, 111, 118, 105, 101, 32, 114, 101, 108, 101, 97, 115, 101, 100, 32, 105, 110, 32, 50, 48, 49, 50, 46, 32, 73, 116, 32, 105, 115, 32, 116, 104, 101, 32, 102, 105, 110, 97, 108, 32, 112, 97, 114, 116, 32, 111, 102, 32, 67, 104, 114, 105, 115, 116, 111, 112, 104, 101, 114, 32, 78, 111, 108, 97, 110, 8217, 115, 32, 68, 97, 114, 107, 32, 75, 110, 105, 103, 104, 116, 32, 116, 114, 105, 108, 111, 103, 121, 44, 32, 102, 111, 108, 108, 111, 119, 105, 110, 103, 32, 66, 97, 116, 109, 97, 110, 32, 66, 101, 103, 105, 110, 115, 32, 97, 110, 100, 32, 84, 104, 101, 32, 68, 97, 114, 107, 32, 75, 110, 105, 103, 104, 116, 46, 32, 84, 104, 101, 32, 102, 105, 108, 109, 32, 115, 116, 97, 114, 115, 32, 67, 104, 114, 105, 115, 116, 105, 97, 110, 32, 66, 97, 108, 101, 32, 97, 115, 32, 66, 114, 117, 99, 101, 32, 87, 97, 121, 110, 101, 47

### Step 2: Write a function to count the frequency of the adjacent pairs of characters

In [ ]:
# 1) Count all adjacent pairs in our current sequence 'ids'.

def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts

stats = get_stats(ids)
print(stats)


{(84, 104): 8, (104, 101): 20, (101, 32): 42, (32, 68): 4, (68, 97): 4, (97, 114): 9, (114, 107): 4, (107, 32): 5, (32, 75): 5, (75, 110): 4, (110, 105): 4, (105, 103): 7, (103, 104): 5, (104, 116): 5, (116, 32): 14, (32, 82): 2, (82, 105): 2, (105, 115): 16, (115, 101): 3, (101, 115): 12, (115, 32): 39, (32, 105): 14, (32, 97): 31, (97, 32): 9, (32, 115): 15, (115, 117): 4, (117, 112): 1, (112, 101): 3, (101, 114): 10, (114, 104): 1, (114, 111): 3, (111, 32): 10, (32, 109): 10, (109, 111): 7, (111, 118): 8, (118, 105): 7, (105, 101): 9, (32, 114): 4, (114, 101): 9, (101, 108): 4, (108, 101): 8, (101, 97): 3, (97, 115): 10, (101, 100): 8, (100, 32): 18, (105, 110): 15, (110, 32): 16, (32, 50): 1, (50, 48): 1, (48, 49): 1, (49, 50): 1, (50, 46): 1, (46, 32): 9, (32, 73): 2, (73, 116): 2, (32, 116): 20, (116, 104): 20, (32, 102): 8, (102, 105): 5, (110, 97): 3, (97, 108): 8, (108, 32): 4, (32, 112): 8, (112, 97): 1, (114, 116): 1, (32, 111): 9, (111, 102): 5, (102, 32): 6, (32, 67): 4, (

### Step 3: Select the pair with the highest frequency

In [ ]:
# 2) Select the pair with the highest frequency.
pair = max(stats, key=stats.get)
print(pair)


(101, 32)


### Step 4: Define the new token's ID (ID of the merged token is added to vocabulary)

In [ ]:
# 3) Define the new token's ID as 256 + i.
i = 0
idx = 128 + i
print(idx)

128


In [ ]:
# For readability, decode the original token IDs (pair[0], pair[1]) into characters
    # just for a nice printout. (Assumes these IDs map to ASCII, etc.)
char_pair = (chr(pair[0]), chr(pair[1]))
print(char_pair)

('e', ' ')


### Step 5: Show which pair we are merging

In [ ]:
# Show which pair we are merging.
print(f"merging {pair} ({char_pair[0]}{char_pair[1]}) into a new token {idx}")

merging (101, 32) (e ) into a new token 256


### Step 6: Peform the merge: replace all occurences of the most frequent pair with the new token ID

In [ ]:
# 4) Perform the merge, replacing all occurrences of 'pair' with 'idx'.
def merge(ids, pair, idx):
    newids = []
    i = 0
    while i < len(ids):
        if i < len(ids) - 1 and ids[i] == pair[0] and ids[i + 1] == pair[1]:
            newids.append(idx)
            i += 2
        else:
            newids.append(ids[i])
            i += 1
    return newids

ids = merge(ids, pair, idx)
print(ids)

[84, 104, 256, 68, 97, 114, 107, 32, 75, 110, 105, 103, 104, 116, 32, 82, 105, 115, 101, 115, 32, 105, 115, 32, 97, 32, 115, 117, 112, 101, 114, 104, 101, 114, 111, 32, 109, 111, 118, 105, 256, 114, 101, 108, 101, 97, 115, 101, 100, 32, 105, 110, 32, 50, 48, 49, 50, 46, 32, 73, 116, 32, 105, 115, 32, 116, 104, 256, 102, 105, 110, 97, 108, 32, 112, 97, 114, 116, 32, 111, 102, 32, 67, 104, 114, 105, 115, 116, 111, 112, 104, 101, 114, 32, 78, 111, 108, 97, 110, 8217, 115, 32, 68, 97, 114, 107, 32, 75, 110, 105, 103, 104, 116, 32, 116, 114, 105, 108, 111, 103, 121, 44, 32, 102, 111, 108, 108, 111, 119, 105, 110, 103, 32, 66, 97, 116, 109, 97, 110, 32, 66, 101, 103, 105, 110, 115, 32, 97, 110, 100, 32, 84, 104, 256, 68, 97, 114, 107, 32, 75, 110, 105, 103, 104, 116, 46, 32, 84, 104, 256, 102, 105, 108, 109, 32, 115, 116, 97, 114, 115, 32, 67, 104, 114, 105, 115, 116, 105, 97, 110, 32, 66, 97, 108, 256, 97, 115, 32, 66, 114, 117, 99, 256, 87, 97, 121, 110, 101, 47, 66, 97, 116, 109, 97, 110,

### Step 7: Write all functions together and define number of iterations to run.

Here, we have to select how many merges we do. If we do 20 merges, the vocabulary size increases from 128 to 148.

In [ ]:
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts

def merge(ids, pair, idx):
    newids = []
    i = 0
    while i < len(ids):
        if i < len(ids) - 1 and ids[i] == pair[0] and ids[i + 1] == pair[1]:
            newids.append(idx)
            i += 2
        else:
            newids.append(ids[i])
            i += 1
    return newids

# ---
vocab_size = 148  # the desired final vocabulary size
num_merges = vocab_size - 128
ids = list(tokens)  # copy so we don't destroy the original list

merges = {}  # (int, int) -> int
for i in range(num_merges):
    # 1) Count all adjacent pairs in our current sequence 'ids'.
    stats = get_stats(ids)
    pair = max(stats, key=stats.get)
    idx = 128 + i
    # Decode the characters of the pair for display
    char_pair = (chr(pair[0]), chr(pair[1]))
    print(f"merging {pair} ({char_pair[0]}{char_pair[1]}) into a new token {idx}")
    ids = merge(ids, pair, idx)
    merges[pair] = idx

merging (101, 32) (e ) into a new token 128
merging (115, 32) (s ) into a new token 129
merging (97, 110) (an) into a new token 130
merging (116, 104) (th) into a new token 131
merging (100, 32) (d ) into a new token 132
merging (105, 110) (in) into a new token 133
merging (116, 32) (t ) into a new token 134
merging (32, 115) ( s) into a new token 135
merging (101, 110) (en) into a new token 136
merging (105, 129) (i) into a new token 137
merging (101, 114) (er) into a new token 138
merging (130, 132) () into a new token 139
merging (104, 128) (h) into a new token 140
merging (97, 114) (ar) into a new token 141
merging (114, 101) (re) into a new token 142
merging (46, 32) (. ) into a new token 143
merging (44, 32) (, ) into a new token 144
merging (84, 140) (T) into a new token 145
merging (111, 32) (o ) into a new token 146
merging (111, 118) (ov) into a new token 147


In [ ]:
print("tokens length:", len(tokens))
print("ids length:", len(ids))
print(f"compression ratio: {len(tokens) / len(ids):.2f}X")

tokens length: 1248
ids length: 953
compression ratio: 1.31X


### In class activity: Take any text of your choice and implement the BPE algorithm. Report the compression ratio achieved.

## Using the tiktoken library

In [ ]:
! pip install tiktoken

import tiktoken

# Text to encode and decode
text = "The lion roams in the jungle"

# ─────────────────────────────────────────────────────────────────────────
# 1. GPT-2 Encoding/Decoding
#    Using the "gpt2" encoding
# ─────────────────────────────────────────────────────────────────────────
tokenizer_gpt2 = tiktoken.get_encoding("gpt2")

# Encode: text -> list of token IDs
token_ids_gpt2 = tokenizer_gpt2.encode(text)

# Decode: list of token IDs -> original text (just to verify correctness)
decoded_text_gpt2 = tokenizer_gpt2.decode(token_ids_gpt2)

# We can also get each token string by decoding the IDs one by one
tokens_gpt2 = [tokenizer_gpt2.decode([tid]) for tid in token_ids_gpt2]

print("=== GPT-2 Encoding ===")
print("Original Text: ", text)
print("Token IDs:     ", token_ids_gpt2)
print("Tokens:        ", tokens_gpt2)
print("Decoded Text:  ", decoded_text_gpt2)
print()


=== GPT-2 Encoding ===
Original Text:  The lion roams in the jungle
Token IDs:      [464, 18744, 686, 4105, 287, 262, 20712]
Tokens:         ['The', ' lion', ' ro', 'ams', ' in', ' the', ' jungle']
Decoded Text:   The lion roams in the jungle



In [ ]:
# ─────────────────────────────────────────────────────────────────────────
# 2. GPT-3.5 Encoding
#    Using the encoding_for_model("gpt-3.5-turbo")
# ─────────────────────────────────────────────────────────────────────────
tokenizer_gpt35 = tiktoken.encoding_for_model("gpt-3.5-turbo")

token_ids_gpt35 = tokenizer_gpt35.encode(text)
decoded_text_gpt35 = tokenizer_gpt35.decode(token_ids_gpt35)
tokens_gpt35 = [tokenizer_gpt35.decode([tid]) for tid in token_ids_gpt35]

print("=== GPT-3.5 Encoding ===")
print("Original Text: ", text)
print("Token IDs:     ", token_ids_gpt35)
print("Tokens:        ", tokens_gpt35)
print("Decoded Text:  ", decoded_text_gpt35)
print()


=== GPT-3.5 Encoding ===
Original Text:  The lion roams in the jungle
Token IDs:      [791, 40132, 938, 4214, 304, 279, 45520]
Tokens:         ['The', ' lion', ' ro', 'ams', ' in', ' the', ' jungle']
Decoded Text:   The lion roams in the jungle



In [ ]:
# ─────────────────────────────────────────────────────────────────────────
# 3. GPT-4 Encoding
#    Using the encoding_for_model("gpt-4")
# ─────────────────────────────────────────────────────────────────────────
tokenizer_gpt4 = tiktoken.encoding_for_model("gpt-4")

token_ids_gpt4 = tokenizer_gpt4.encode(text)
decoded_text_gpt4 = tokenizer_gpt4.decode(token_ids_gpt4)
tokens_gpt4 = [tokenizer_gpt4.decode([tid]) for tid in token_ids_gpt4]

print("=== GPT-4 Encoding ===")
print("Original Text: ", text)
print("Token IDs:     ", token_ids_gpt4)
print("Tokens:        ", tokens_gpt4)
print("Decoded Text:  ", decoded_text_gpt4)